In [1]:
import xml.etree.ElementTree as Xet
import pandas as pd
import re

In [2]:
prefix = "test:"

In [3]:
xmlparse = Xet.parse("./Chataignier_Bourran-PF.xml")

In [4]:
root = xmlparse.getroot()

In [5]:
# TODO un peu d'opti?
def extract(elem, d, parents, n, uri, ignore):
    if "{" in elem.tag:
        s = re.sub('{[^}]+}', '', elem.tag).lower()
    else :
        s = elem.tag.lower()
        
    if s not in d.keys():
        d[s] = pd.DataFrame()
    
    newrow = {}

    for k in elem.attrib.keys():
        if "{" in k:
            s2 = re.sub('{[^}]+}', '', k).lower()
        else :
            s2 = k.lower()
        if s2 not in ignore:
            newrow[s2] = elem.attrib[k]
        
            if "/" in newrow[s2]:
                
                l = newrow[s2].split(" ")

                for i in range(len(l)):
                    tmp_l = l[i].split("/")[1:]
                    tmp_l = [x if any([y.isdigit() for y in x]) else x + ".0" for x in tmp_l]
                    tmp_l = "/".join(tmp_l).lower()
                    l[i] = prefix + "/" + tmp_l
                newrow[s2] = l

    newrow.update(parents)
    newrow['uri'] = uri

    d[s] = d[s].append(newrow, ignore_index=True)

    parents.update({s : n})

    n = 0
    t = ""
    for child in elem:
        if not any([x in child.tag.lower() for x in ignore]):
            if child.tag.lower() != t:
                t=child.tag.lower()
                if "{" in t:
                    t = re.sub('{[^}]+}', '', t)
                else :
                    t = t
                n=0
            s = uri + "/@{}.{}".format(t, n)
            extract(child, d, parents.copy(), n, s, ignore)
            n += 1

In [6]:
d = {}
n = 0
parents = {}
# list of tags to ignore
ignore = ["codeetat", "codesetats", "structuregraphique", "droitsacces", "droitutilisateurauthentifie", "droitutilisateurautre"]
for elem in root:
    if not any([x in elem.tag.lower() for x in ignore]):
        extract(elem, d, parents.copy(), n, prefix + "/" + str(n), ignore)
        n += 1

In [7]:
d.keys()

dict_keys(['plateforme', 'dispositifs', 'blocs', 'parcellesunitaire', 'individus', 'protocoles', 'facteurs', 'modalites', 'traitements', 'saisie', 'sessions', 'mesurevariables', 'variables', 'echelle', 'notations', 'utilisateur', 'projetdesaisie'])

In [8]:
d['traitements']

,combinaisons,nom,nomcourt,nbrepetitions,plateforme,dispositifs,protocoles,uri
0,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,16_1,1,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
1,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,1000,2,232,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
2,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,36_1,3,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
3,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,27_1,4,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
4,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,13_1,5,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
...,...,...,...,...,...,...,...,...
654,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,8_7,655,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
655,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,11_31,656,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
656,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,13_102,657,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...
657,[test:/0/@dispositifs.0/@protocoles.0/@facteur...,4_40,658,1,0.0,0.0,0.0,test:/0/@dispositifs.0/@protocoles.0/@traiteme...


In [9]:
d['traitements'].iloc[0,0][0]

'test:/0/@dispositifs.0/@protocoles.0/@facteurs.0/@modalites.0'

In [10]:
d['modalites']['uri'].iloc[0]

'test:/0/@dispositifs.0/@protocoles.0/@facteurs.0/@modalites.0'

In [11]:
for k in d.keys():
    d[k].to_csv("./script_out/{}_V2.csv".format(k), index=False)

## Check for match

In [12]:
for k in d.keys():
    for col in d[k].columns:
        if any(d[k][col].astype(str).str.contains("/")) and col != "uri":
            print(k, ":", col)
            for row in d[k][col]:
                for to_match in row:
                    categ = to_match.split("/")[-1]
                    if categ.isdigit():
                        pass
                    elif not (d[categ.split(".")[0][1:]]["uri"] == to_match).any():
                        print(to_match, False)

plateforme : createur
dispositifs : createur
dispositifs : gestionnaires
parcellesunitaire : traitement
protocoles : createur
traitements : combinaisons
saisie : projetdesaisie
saisie : plateforme
sessions : experimentateur
mesurevariables : variable
mesurevariables : objetmetier
projetdesaisie : createur
projetdesaisie : plateforme
projetdesaisie : dispositifs


In [13]:
d.keys()

dict_keys(['plateforme', 'dispositifs', 'blocs', 'parcellesunitaire', 'individus', 'protocoles', 'facteurs', 'modalites', 'traitements', 'saisie', 'sessions', 'mesurevariables', 'variables', 'echelle', 'notations', 'utilisateur', 'projetdesaisie'])

In [14]:
d['dispositifs']

,numero,nom,datecreation,createur,gestionnaires,source,etat,datevalidation,plateforme,uri
0,1,ChataignierBourran,2015-07-03T12:02:20.078+0200,[test:/4],"[test:/5, test:/2, test:/4]",essai.csv,valide,2015-07-03T12:06:17.230+0200,0.0,test:/0/@dispositifs.0
